# Prelims

<b style="color: red">To use this notebook, go to Runtime > Change Runtime Type and select GPU as the hardware accelerator.</b>

This is because tokenizer will use .cuda to process input batches in parallel.

# Setup
(No need to read)

In [ ]:
%%capture
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [ ]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [ ]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [ ]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

# Analyze GPT-2-Small
80M parameter model

## Loading and Running Models

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


# adj

In [ ]:
example_prompt = "Bob is blue. Mary is red. Bob is"
example_answer = " blue"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'Bob', ' is', ' blue', '.', ' Mary', ' is', ' red', '.', ' Bob', ' is']
Tokenized answer: [' blue']


Performance on answer token:
Rank: 0        Logit: 14.46 Prob: 10.27% Token: | blue|

Top 0th token. Logit: 14.46 Prob: 10.27% Token: | blue|
Top 1th token. Logit: 14.25 Prob:  8.35% Token: | white|
Top 2th token. Logit: 14.18 Prob:  7.74% Token: | a|
Top 3th token. Logit: 13.76 Prob:  5.11% Token: | green|
Top 4th token. Logit: 13.49 Prob:  3.90% Token: | red|
Top 5th token. Logit: 13.36 Prob:  3.43% Token: | yellow|
Top 6th token. Logit: 13.36 Prob:  3.42% Token: | black|
Top 7th token. Logit: 13.19 Prob:  2.88% Token: | purple|
Top 8th token. Logit: 12.94 Prob:  2.25% Token: | pink|
Top 9th token. Logit: 12.68 Prob:  1.72% Token: | the|


Ranks of the answer tokens: [(' blue', 0)]

In [ ]:
def logits_to_ave_logit_diff(logits, answer_tokens, per_prompt=False):
        # Only the final logits are relevant for the answer
        final_logits = logits[:, -1, :]
        answer_logits = final_logits.gather(dim=-1, index=answer_tokens)
        answer_logit_diff = answer_logits[:, 0] - answer_logits[:, 1]
        if per_prompt:
            return answer_logit_diff
        else:
            return answer_logit_diff.mean()

def residual_stack_to_logit_diff(residual_stack: Float[torch.Tensor, "components batch d_model"], cache: ActivationCache, 
                                 logit_diff_directions) -> float:
        scaled_residual_stack = cache.apply_ln_to_stack(residual_stack, layer = -1, pos_slice=-1)
        return einsum("... batch d_model, batch d_model -> ...", scaled_residual_stack, logit_diff_directions)/len(prompts)

def make_attn_heads(prompts, answers):
    # used to get answer_residual_directions
    answer_tokens = []
    for answer in answers:
        correct_token = model.to_single_token(answer[0])
        incorrect_token = model.to_single_token(answer[1])
        answer_tokens.append((correct_token, incorrect_token))
    # answer_tokens = torch.tensor(answer_tokens).cuda()  # if many inputs
    answer_tokens = torch.tensor(answer_tokens)

    tokens = model.to_tokens(prompts, prepend_bos=True)
    # tokens = tokens.cuda() # Move the tokens to the GPU
    original_logits, cache = model.run_with_cache(tokens) # Run the model and cache all activations

    # print("Per prompt logit difference:", logits_to_ave_logit_diff(original_logits, answer_tokens, per_prompt=True))
    # original_average_logit_diff = logits_to_ave_logit_diff(original_logits, answer_tokens)
    # print("Average logit difference:", logits_to_ave_logit_diff(original_logits, answer_tokens).item())

    # logit_diff_directions used to get per_head_logit_diffs
    answer_residual_directions = model.tokens_to_residual_directions(answer_tokens)
    print("Answer residual directions shape:", answer_residual_directions.shape)
    logit_diff_directions = answer_residual_directions[:, 0] - answer_residual_directions[:, 1]
    print("Logit difference directions shape:", logit_diff_directions.shape)

    # accumulated_residual, labels = cache.accumulated_resid(layer=-1, incl_mid=True, pos_slice=-1, return_labels=True)
    # logit_lens_logit_diffs = residual_stack_to_logit_diff(accumulated_residual, cache)
    # line(logit_lens_logit_diffs, x=np.arange(model.cfg.n_layers*2+1)/2, hover_name=labels, title="Logit Difference From Accumulate Residual Stream")

    per_head_residual, labels = cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)
    per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, cache, logit_diff_directions)
    per_head_logit_diffs = einops.rearrange(per_head_logit_diffs, "(layer head_index) -> layer head_index", layer=model.cfg.n_layers, head_index=model.cfg.n_heads)
    imshow(per_head_logit_diffs, labels={"x":"Head", "y":"Layer"}, title="Logit Difference From Each Head")
    return per_head_logit_diffs

prompts = [
    "Bob is blue. Mary is red. Bob is",
]
# List of answers, in the format (correct, incorrect)
answers = [
    (" blue", " red"),
]

per_head_logit_diffs = make_attn_heads(prompts, answers)

Answer residual directions shape: torch.Size([1, 2, 768])
Logit difference directions shape: torch.Size([1, 768])
Tried to stack head results when they weren't cached. Computing head results now


In [ ]:
top_indices = get_top_indices(per_head_logit_diffs, 10)
for row, col in top_indices:
    print(f"L={row}, H={col}")
top_indices

L=8, H=6
L=10, H=1
L=9, H=6
L=0, H=9
L=5, H=1
L=11, H=0
L=7, H=11
L=5, H=5
L=5, H=0
L=11, H=11


[(8, 6),
 (10, 1),
 (9, 6),
 (0, 9),
 (5, 1),
 (11, 0),
 (7, 11),
 (5, 5),
 (5, 0),
 (11, 11)]